In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir('..')
sys.path.append('src')

In [ ]:
from pathlib import Path
import random

from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib widget 


In [ ]:

import nvidia.dali.ops as ops

In [ ]:
u = ops.random.Uniform(seed=3)

In [ ]:
# @pipeline_def
# def simple_pipeline():
#     jpegs_images, _= fn.readers.file(files=images_files, seed=1234, name="main_reader", num_shards=world_size, shard_id=global_rank)
#     jpegs_masks, _= fn.readers.file(files=masks_files, seed=1234, num_shards=world_size, shard_id=global_rank
#     images= fn.decoders.image(jpegs_images, device='mixed')
#     masks= fn.decoders.image(jpegs_masks, device='mixed')
#     return images, masks

# pipe = simple_pipeline(batch_size=max_batch_size, num_threads=1, device_id=0)
# iter = DALIGenericIterator([pipe], ['data', 'label'], reader_name='main_reader')

In [ ]:
root = Path('input/preprocessed/png1024/')
imgs = list((root / 'images').rglob('*'))
masks = list((root / 'masks').rglob('*'))
len(imgs), len(masks)

In [ ]:
def random_flip(images, probability, seed):
    flip = fn.random.coin_flip(probability=1-probability, seed=seed)
    return fn.flip(images, horizontal=flip)

@pipeline_def
def segmentation_pipe(input_files, crop_size):
    image_files, mask_files = input_files
    seed = random.randint(1, 0x7fffffff)  # get a common seed
    
    enc_images, _ = fn.readers.file(files=image_files, seed=seed)
    enc_masks, _ = fn.readers.file(files=mask_files, seed=seed)
    images = fn.decoders.image(enc_images, device="mixed")
    masks = fn.decoders.image(enc_masks, device="mixed")
    #return images, masks
                                                                                                                                  
    crop = ops.Crop(crop=crop_size, output_dtype=types.FLOAT)                                                                                                       
    uniform = ops.random.Uniform(range=(0.0, 1.0), seed=seed)
    
    images = crop(images, crop_pos_x=uniform(), crop_pos_y=uniform(), seed=seed)                                                                                    
    masks = crop(masks, crop_pos_x=uniform(), crop_pos_y=uniform(), seed=seed) 
    
    # area_min = area_max = 256/1024
    # images = fn.random_resized_crop(
    #     images,
    #     size=crop_size,
    #     random_area=[area_min, area_max],
    #     random_aspect_ratio=[0.9, 1.1],
    #     seed=seed)  # the seed
    # masks = fn.random_resized_crop(
    #     masks,
    #     size=crop_size,
    #     interp_type = types.INTERP_NN,
    #     random_area=[area_min, area_max],
    #     random_aspect_ratio=[0.9, 1.1],
    #     seed=seed)  # same seed
    
    # image_patches = random_flip(image_patches, .5, seed)
    # image_patches = random_flip(image_patches, .5, seed)
    
    return images, masks

In [ ]:
p = segmentation_pipe((imgs, masks), crop_size=256, batch_size=4, num_threads=4, device_id=0,)

In [ ]:
train_data = DALIGenericIterator(p, ['img_data', 'ann_data'], auto_reset=True, size=len(imgs))

In [ ]:
len(train_data)

In [ ]:
for j, i in enumerate(train_data):
    if j % 50 == 0:
        print(j, i[0]['img_data'].shape)

In [ ]:
import torch

In [ ]:
train_data.reset()

In [ ]:
t = next(train_data)

In [ ]:
dl = torch.utils.data.DataLoader(train_data, 1, )

In [ ]:
for x in dl:
    break

In [ ]:
x.shape

In [ ]:
for i in train_data:
    break
len(i)

xb = i[0]['img_data']
yb = i[0]['ann_data']
xb.shape, yb.shape

x = xb[0]
y = yb[0]*255
x[...,0] = y[...,0]

plt.figure()
plt.imshow(x.cpu())

In [ ]:
plt.figure()
plt.imshow(xb[0].cpu())

In [ ]:


# for i, data in enumerate(train_data):
#     x, y = data[0]['data'], data[0]['label']
#     pred = model(x)
#     loss = loss_func(pred, y)
#     backward(loss, model)